##### Prueba glounts 
https://forecastegy.com/posts/multiple-time-series-forecasting-with-deepar-in-python/

cosas pro buenas definiciones 
https://medium.com/@gabri_orl/a-primer-on-gluonts-a8bff64b1096

Por otro lado, obtenemos datos historicos semanales del tipo de cambio Dolar Blue y el contado con liquidacion para evaluar cual de ellos posee mayor correlacion con las series

Cotizacion Dolar Blue obtenido de: https://es.investing.com/currencies/usd-arsb-historical-data

In [48]:
import pandas as pd
import numpy as np

In [49]:
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

In [50]:
df_impo_1= pd.read_csv('Data/Modelado/impo 1 - USA FLAT.csv')
df_impo_2= pd.read_csv('Data/Modelado/impo 2 - ORIENTE UPS.csv')
df_impo_3= pd.read_csv('Data/Modelado/impo 3 - CHINA LATIN LOGISTIC.csv')
df_impo_4= pd.read_csv('Data/Modelado/impo 4 - EUROPA UPS.csv')

In [51]:
len(df_impo_1)

209

In [52]:
def unificar_df_ar(lista_df):
    
    lista=[]
    for df in lista_df: 
        
        df= df[['ds', 'family', 'y']]
        lista.append(df)
    
    df_final = pd.concat(lista, ignore_index=True)
    df_final = df_final[['ds', 'family', 'y']]
    df_final.style.format({"ds": lambda t: t.strftime("%d-%m-%Y")})
    
    return df_final    

In [53]:
df_impo_1['family']= 'impo_1'
df_impo_2['family']='impo_2'
df_impo_3['family']='impo_3'

In [54]:
lista_dataframe= [df_impo_1, df_impo_2, df_impo_3]
df= unificar_df_ar(lista_dataframe)

In [55]:
df['ds']= pd.to_datetime(df['ds'])

In [56]:
df.head()

,ds,family,y
0,2019-01-06,impo_1,1950.0
1,2019-01-13,impo_1,2402.0
2,2019-01-20,impo_1,2782.0
3,2019-01-27,impo_1,3263.0
4,2019-02-03,impo_1,3843.0


In [57]:
train = df.loc[df['ds'] < '2022-11-01']
valid = df.loc[df['ds'] >= '2022-11-01']

In [58]:
len(valid)

27

In [59]:
from gluonts.dataset.pandas import PandasDataset

train_ds = PandasDataset.from_long_dataframe(train, target='y', item_id='family', 
                                       timestamp='ds', freq='W')

In [60]:
from gluonts.torch.model.deepar import DeepAREstimator

estimator = DeepAREstimator(freq='W', #la frecuencia de los datos es semanal por lo que el modelo debe ser semanal tambien
                            context_length=4, # el modelo va a usar las ultimas 4 semanas para predecir la siguiente 
                            prediction_length=10, # va a predecir las proximas 10 semanas 
                            num_layers=3, #el modelo posee tres capas con un default de 40 nodos por capa 
                            dropout_rate= 0.2, #seteo de 20% de las units en una layer en cero de forma random, el default es 0.1
                            trainer_kwargs={ 'max_epochs':30}) #en vez de poner un early stopping defino un max epoch de 30

predictor = estimator.train(train_ds, num_workers=2)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Bravo15\anaconda3\envs\modelling-torch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:71: PossibleUserWarning:

You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.


  | Name  | Type        | Params | In sizes | Out sizes   
----------------------------------------------------------------
0 | model | DeepARModel | 36.9 K | ?        | [1, 100, 10]
----------------------------------------------------------------
36.9 K    Trainable params
0         Non-trainable params
36.9 K    Total params
0.148     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 7.13343 (best 7.13343), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_69\\checkpoints\\epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 6.63537 (best 6.63537), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_69\\checkpoints\\epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 6.44533 (best 6.44533), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_69\\checkpoints\\epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' was not in top 1
Epoch 4, global step 250: 'train_loss' reached 6.21100 (best 6.21100), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_69\\checkpoints\\epoch=4-step=250.ckpt' as top 1
Epoch 

In [61]:
pred = list(predictor.predict(train_ds))

In [62]:
all_preds = list()
for item in pred:
    family = item.item_id
    p = item.samples.mean(axis=0)
    p10 = np.percentile(item.samples, 10, axis=0)
    p90 = np.percentile(item.samples, 90, axis=0)
    p25 = np.percentile(item.samples, 25, axis=0)
    p75 = np.percentile(item.samples, 75, axis=0)
    dates = pd.date_range(start=item.start_date.to_timestamp(), periods=len(p), freq='W')
    family_pred = pd.DataFrame({'ds': dates, 
                                'family': family,
                                'pred': p,
                                'p25': p25,
                                'p75': p75,
                                'p10': p10, 
                                'p90': p90})
    all_preds += [family_pred]
all_preds = pd.concat(all_preds, ignore_index=True)
all_preds = all_preds.merge(valid, on=['ds', 'family'], how='left')

all_preds.dropna(inplace=True)

wmape(all_preds['y'], all_preds['pred'])

0.4233193068858342

In [63]:
all_preds.head(20)

,ds,family,pred,p25,p75,p10,p90,y
0,2022-11-06,impo_1,3.534516e+03,3.016410e+03,4.136897e+03,2.586153e+03,4.603896e+03,3067.0
1,2022-11-13,impo_1,4.233324e+03,3.488875e+03,4.805924e+03,3.062530e+03,5.821342e+03,2557.0
2,2022-11-20,impo_1,4.202205e+03,3.520620e+03,5.008361e+03,2.785067e+03,5.754860e+03,1651.0
3,2022-11-27,impo_1,4.280624e+03,3.541265e+03,5.053482e+03,2.839637e+03,5.771206e+03,2538.0
4,2022-12-04,impo_1,4.237413e+03,3.199849e+03,5.216983e+03,2.418987e+03,5.923500e+03,2321.0
5,2022-12-11,impo_1,4.329830e+03,3.405455e+03,5.129216e+03,2.767741e+03,5.799688e+03,4285.0
6,2022-12-18,impo_1,4.498466e+03,3.803968e+03,5.296319e+03,2.673329e+03,6.034997e+03,4084.0
7,2022-12-25,impo_1,4.193477e+03,3.207196e+03,4.933800e+03,2.747939e+03,6.013289e+03,3274.0
8,2023-01-01,impo_1,3.903644e+03,2.978430e+03,4.917489e+03,2.388354e+03,5.630591e+03,2813.0
10,2022-11-06,impo_2,2.508251e+02,1.269317e+02,4.208515e+02,-1.102715e+02,6.016490e+02,446.8


In [64]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
import plotly.express as px
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.DARKLY])



dropdown = dcc.Dropdown(
    id="ticker",
    options=[{"label": family, "value": family} for family in df["family"].unique()],
    value="impo_1",
    clearable=False
)

app.layout = html.Div([
    html.H4('Deep AR Resultados'),
    html.P("Seleccionar motivo:"),
    dropdown,
    dcc.Graph(id="time-series-chart"),
])


@app.callback(
    Output("time-series-chart", "figure"), 
    Input("ticker", "value"))

def display_time_series(ticker):
    df_pivot = df.pivot(index='ds', columns='family', values='y')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    
    fig = px.line(df_pivot, x='ds', y=ticker, title=ticker, template= "plotly_dark")
    
    #bandas de percentiles 
    p_ = all_preds.loc[all_preds['family'] == ticker]
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p10'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p10'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p90'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p90', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p25'], mode='lines', line=dict(color='orange'), name='p25'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p75'], mode='lines', line=dict(color='orange'), name='p75', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['pred'], mode='lines', line=dict(color='orange'), name='Forecast'))
    return fig


if __name__ == "__main__":
    app.run_server(debug=True)


In [65]:
### Deep AR con serie de la cotizacion del dolar Blue

In [66]:
df_cot_dolar_blue= pd.read_csv('Data/Datos históricos USD_ARSB.csv')

In [67]:
df_cot_dolar_blue.head()

,Fecha,Último,Apertura,Máximo,Mínimo,Vol.,% var.
0,29.01.2023,"377,00","389,00","377,00","377,00",NaN,"-1,82%"
1,22.01.2023,"384,00","377,00","384,00","384,00",NaN,"2,67%"
2,15.01.2023,"374,00","368,00","374,00","374,00",NaN,"1,91%"
3,08.01.2023,"367,00","356,00","367,00","367,00",NaN,"4,26%"
4,01.01.2023,"352,00","350,00","352,00","352,00",NaN,"2,33%"


In [68]:
df_cot_dolar_blue.dtypes

Fecha        object
Último       object
Apertura     object
Máximo       object
Mínimo       object
Vol.        float64
% var.       object
dtype: object

In [69]:
df_cot_dolar_blue['Fecha'] = pd.to_datetime(df_cot_dolar_blue['Fecha'], format='%d.%m.%Y')

In [70]:
df_cot_dolar_blue= df_cot_dolar_blue[['Fecha', 'Último']]

In [71]:
## Dataframe unificado
df.head()

,ds,family,y
0,2019-01-06,impo_1,1950.0
1,2019-01-13,impo_1,2402.0
2,2019-01-20,impo_1,2782.0
3,2019-01-27,impo_1,3263.0
4,2019-02-03,impo_1,3843.0


Para cada fecha tengo que mapear la cotizacion del dolar de esa semana, para eso como ya tengo los dos datasets en el mismo formato mapeo con un dic el valor de la cotizacion de cada semana usando las fechas 

In [72]:
df_merged = df.merge(df_cot_dolar_blue, left_on='ds', right_on='Fecha', how='left')

# Renombra la columna 'Último' a 'cotizacion' y elimina la columna 'fecha'
df_merged = df_merged.rename(columns={'Último': 'cotizacion'}).drop('Fecha', axis=1)

In [73]:
df= df_merged
df.head()

,ds,family,y,cotizacion
0,2019-01-06,impo_1,1950.0,"39,00"
1,2019-01-13,impo_1,2402.0,"38,88"
2,2019-01-20,impo_1,2782.0,"38,12"
3,2019-01-27,impo_1,3263.0,"37,62"
4,2019-02-03,impo_1,3843.0,"37,38"
